# Lab 04: Linear Regression and Causality

This lab asks you to do two tasks:

First, to estimate the effect of Right Hearth
Catheterization (RHC)--inserting a tiny hollow tube along the arterial into
the right side of your heart.  It is often done for diagnostical
purposes (it allows to get different measures right in the heart) and
it's usually considered safe.

We will use a dataset about RHC for critically ill patients and see
if RHC is related to increased death rate.  The dataset is downloaded
from [Vanderbilt
Biostats](http://biostat.mc.vanderbilt.edu/wiki/Main/DataSets) and
more information is available at
[http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/rhc.html](http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/rhc.html).

Second, you have to create (or re-create) the design matrices of the
first problem, analyze their properties, and compute the regression
coefficients of the first part "manually", i.e. by a simple matrix
expression on computer.


## 1 Get Ready

Load the data.  A tab-separated version is available on canvas (files/data).

1. How many observation and variables do we have?
2. The most important variables we use below are _death_ (patient
death recorded/not recorded) and
_swang1_ (rhc performed/not performed).  How are these variables coded?

In [27]:
import numpy as np
import pandas as pd
from scipy import stats # ANOVA
from sklearn.linear_model import LinearRegression

In [20]:
rhc_df= pd.read_csv('rhc.csv')
rhc_df.head()

,Unnamed: 0,cat1,cat2,ca,sadmdte,dschdte,dthdte,lstctdte,death,cardiohx,...,meta,hema,seps,trauma,ortho,adld3p,urin1,race,income,ptid
0,1,COPD,NaN,Yes,11142,11151.0,NaN,11382,No,0,...,No,No,No,No,No,0.0,NaN,white,Under $11k,5
1,2,MOSF w/Sepsis,NaN,No,11799,11844.0,11844.0,11844,Yes,1,...,No,No,Yes,No,No,NaN,1437.0,white,Under $11k,7
2,3,MOSF w/Malignancy,MOSF w/Sepsis,Yes,12083,12143.0,NaN,12400,No,0,...,No,No,No,No,No,NaN,599.0,white,$25-$50k,9
3,4,ARF,NaN,No,11146,11183.0,11183.0,11182,Yes,0,...,No,No,No,No,No,NaN,NaN,white,$11-$25k,10
4,5,MOSF w/Sepsis,NaN,No,12035,12037.0,12037.0,12036,Yes,0,...,No,No,No,No,No,NaN,64.0,white,Under $11k,11


In [21]:
# How many observation and variables do we have?
# 5735 observations and 63 variables
display(rhc_df.shape)

# The most important variables we use below are death (patient death recorded/not recorded) 
# and swang1 rhc performed/not performed). How are these variables coded?
display(type(rhc_df.death))
display(type(rhc_df.swang1))

(5735, 63)

pandas.core.series.Series

pandas.core.series.Series

## 2 Cross-Sectional Estimator

Estimate, using linear regression, how is the RHC related to mortality.

We start easy: let's just find the relationship between recorded death
and rhc.  The important variables are

* _death_: patient death recorded/not recorded
* _swang1_: rhc performed/not performed

This is called "cross-sectional estimator", we just compare
cross-section of individuals who received and did not receive RHC.

Obviously, this is a very crude measure because the hospitals track
patients over different time periods, and if contact is lost early,
the death may not be recorded.  Another obvious problem is that the
patients have very different medical conditions, a factor that
most likely plays a role in the decision whether to perform RHC.


In [64]:
rhc_df["death"] = rhc_df["death"].replace("Yes", 1)
rhc_df["death"] = rhc_df["death"].replace("No", 0)
rhc_df["swang1"] = rhc_df["swang1"].replace("RHC", 1)
rhc_df["swang1"] = rhc_df["swang1"].replace("No RHC", 0)

X = rhc_df[["death"]]
Y = rhc_df[["swang1"]]

reg = LinearRegression()
mdl = reg.fit(X, Y)

m = mdl.coef_[0]
b = mdl.intercept_
print("formula: y = {0}x + {1}".format(m, b))
print('Score: ', reg.score(X, Y))

formula: y = [ 0.05250157]x + [ 0.34674615]
Score:  0.00266293985003


Comment your results.
### y = 0.052501566256730345x + 0.34674615002483866

## 3 Address some of the issues

Now let's try to address some of the issues with the estimator above.
The dataset includes many other patient descriptors.


### 3.1 Let's include 'age' and 'sex'.
How are these coded?


In [49]:
display(type(rhc_df.age))
display(type(rhc_df.sex))

pandas.core.series.Series

pandas.core.series.Series

What do you find?



### 3.2 Include age and sex in the regression

Now allow the death to depend on gender and age, on top of that it may
depend on rhc.  Note that it may not just depend on age in a linear
fashion but in a much more complex way, so include not just $age$ but
also $age^2$ and $age^3$ as explanatory variables.

In [61]:
rhc_df["sex"] = rhc_df["sex"].replace("Female", 1)
rhc_df["sex"] = rhc_df["sex"].replace("Male", 0)

X = rhc_df[["death"]]
Y = rhc_df.as_matrix(["age", "sex"]).astype("float32")

reg = LinearRegression()
mdl =reg.fit(X, Y)

m = mdl.coef_[0]
b = mdl.intercept_
print("formula: y = {0}x + {1}".format(m, b))
print('Score: ', reg.score(X, Y))

formula: y = [ 7.51575451]x + [ 56.49817795   0.45653254]
Score:  0.0462061437395


Comment your results
### y = [ 7.51575451]x + [ 56.49817795   0.45653254]


## 4 Design matrices of the models

Each linear model (and many other models) have associated _design
matrices_.  Design matrix is the matrix of all your explanatory
variables (all x-s) in the final numeric matrix form.  This includes
* adding a constant column
* converting categorical/non-numeric variables into suitable numeric
variables

You next task is to create/extract the design matrices of both of the
models above, investigate their numeric properties (condition
numbers), and solve the linear regression problem in matrix form.

We did not have time in the class to talk about it, but there is a
closed-form solution for the linear regression problem:  beta =
$(X'X)^{-1} X'y$.  Compute this solution and compare with the regression
package output above.


### 4.1 First model

#### 4.1.1 create the design matrix of it, X.

Depending on the way you solved your problem, you may already have
created it.  Depending on the way you solved the problem above, you
may be able to extract it from the existing model.  You may also redo it
manually here.  Remember:
* include the constant term!
* design matrix must be a _matrix_, not data frame or something else.

In [67]:
matrix = np.matrix(rhc_df[["death", "age", "sex"]])
matrix

matrix([[  0.     ,  70.25098,   0.     ],
        [  1.     ,  78.17896,   1.     ],
        [  0.     ,  46.09198,   1.     ],
        ..., 
        [  1.     ,  80.48499,   0.     ],
        [  1.     ,  67.37897,   0.     ],
        [  1.     ,  54.66397,   1.     ]])

#### 4.1.2 Compute the condition number of X`X.

You may choose whatever definition you like, but please report what
are you using.

In [69]:
XtX = (matrix.T).dot(matrix)
XtX

matrix([[  3.72200000e+03,   2.38259857e+05,   1.62400000e+03],
        [  2.38259857e+05,   2.31994357e+07,   1.57798257e+05],
        [  1.62400000e+03,   1.57798257e+05,   2.54300000e+03]])

#### 4.1.3 Compute your regression coefficients using the formula above.

Note: you also need your outcome variable $y$ in numeric matrix
form. 

In [77]:
inverse = np.linalg.inv(XtX)
coe = inverse.dot(matrix.T).dot(matrix[:,0])
coe

matrix([[  1.00000000e+00],
        [ -3.27293531e-18],
        [  1.86482774e-17]])

#### 4.1.4 Compare your coefficients here with the OLS results above.

In [78]:
coe

matrix([[  1.00000000e+00],
        [ -3.27293531e-18],
        [  1.86482774e-17]])